In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings("ignore")

plt.style.use("seaborn-whitegrid")


In [ ]:

# Load dataset
df = pd.read_excel("Датасет по КРС.xlsx")
df["Период"] = pd.to_datetime(df["Период"], format="%Y-%m")
df = df.rename(columns={"КРС": "Объем убоя: КРС"})
target = "Объем убоя: КРС"


In [ ]:

regions = df["Регион"].unique()
results = []

for region in regions:
    region_df = df[df["Регион"] == region].copy()
    region_df = region_df.drop(columns="Регион")

    train = region_df[region_df["Период"] < "2024-01"]
    test = region_df[(region_df["Период"] >= "2024-01") & (region_df["Период"] < "2024-04")]

    # Holt-Winters
    hw_model = ExponentialSmoothing(train[target], seasonal='add', seasonal_periods=12).fit()
    hw_forecast = hw_model.forecast(len(test))

    # SARIMA
    sarima_model = SARIMAX(train[target], order=(1,1,1), seasonal_order=(1,1,1,12)).fit(disp=False)
    sarima_forecast = sarima_model.forecast(len(test))

    # Prophet
    prophet_train = train.rename(columns={"Период": "ds", target: "y"})
    prophet_model = Prophet()
    prophet_model.fit(prophet_train)
    future = prophet_model.make_future_dataframe(periods=len(test), freq="M")
    prophet_forecast_df = prophet_model.predict(future)
    prophet_forecast = prophet_forecast_df.iloc[-len(test):]["yhat"].values

    # Evaluation
    y_true = test[target].values
    for name, forecast in zip(["Holt-Winters", "SARIMA", "Prophet"], [hw_forecast, sarima_forecast, prophet_forecast]):
        rmse = np.sqrt(mean_squared_error(y_true, forecast))
        mae = mean_absolute_error(y_true, forecast)
        mape = np.mean(np.abs((y_true - forecast) / y_true)) * 100
        results.append({
            "Регион": region,
            "Метод": name,
            "RMSE": rmse,
            "MAE": mae,
            "MAPE (%)": mape,
        })


In [ ]:

results_df = pd.DataFrame(results)
best_models = results_df.loc[results_df.groupby("Регион")["MAPE (%)"].idxmin()].reset_index(drop=True)
print("Лучшие модели по регионам:")
display(best_models)
